In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import requests
import time
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup

# celebrity, famous, actor, actress, soccer, baseball, basketball, football, male, female, woman, man, guy, girl

In [9]:
# Webscraper
url = 'https://www.bing.com/images/search?q=male+face&go=Search&qs=ds&form=QBIR&qft=+filterui%3Alicense-L2_L3_L4_L5_L6_L7'

In [15]:
def scroll(driver, timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
    
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            
            # If heights are the same it will exit the function
            break
        
        last_height = new_height

In [16]:
# Your options may be different
options = Options()
options.set_preference('permissions.default.image', 2)
options.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', False)

def all_links(url):
    
    # Setup the driver. This one uses firefox with some options and a path to the geckodriver
    driver = webdriver.Firefox(options=options, executable_path='D:/Users/bengriffith/Desktop/geckodriver.exe')

    # implicitly_wait tells the driver to wait before throwing an exception
    driver.implicitly_wait(30)

    # driver.get(url) opens the page
    driver.get(url)
    
    # This starts the scrolling by passing the driver and a timeout
    scroll(driver, 5)
    
    # Once scroll returns bs4 parsers the page_source
    soup_a = BeautifulSoup(driver.page_source, 'lxml')
    
    # Them we close the driver as soup_a is storing the page source
    driver.close()

    # Empty array to store the links
    links = {}
    
    # Counter
    link_counter = 0

    # Looping through all the a elements in the page source
    for link in soup_a.find_all('a', {'class': 'iusc'}):
        # link.get('href') gets the href/url out of the a element
        # links.append(link.get('href'))
        links[link_counter] = link.get('m')
        link_counter += 1
        
    for key, value in links.items():
        links[key] = json.loads(value)
        
    return links

In [25]:
# Empty array to store the links
links = []

# Looping through all the a elements in the page source
for link in soup_a.find_all('a', {'class': 'iusc'}):
    # link.get('href') gets the href/url out of the a element
    links.append(link.get('href'))

In [70]:
murl = {}
i = 0
for link in soup_a.find_all('a', {'class': 'iusc'}):
    murl[i] = link.get('m')
    i += 1

In [72]:
murl_dict = {}
for key, value in murl.items():
    murl_dict[key] = json.loads(value)

In [75]:
for key, value in murl_dict.items():
    print(value['murl'])

https://www.eurekalert.org/multimedia/pub/web/169764_web.jpg
http://www.malemodelscene.net/wp-content/gallery/102012/ben-irvine-alex-evans-10.jpg
https://i.pinimg.com/originals/5b/c6/e6/5bc6e6b23f963cb859ac7aa748029a78.png
https://s-media-cache-ak0.pinimg.com/originals/d8/46/a4/d846a423ce2f9ec6ebd0d1a3795d9646.jpg
http://www.thefashionisto.com/wp-content/uploads/2011/07/mt-ff2.jpg
https://i.pinimg.com/736x/1f/d2/92/1fd2929604b222404ec8eecfdafec9f4--tumblr-boys-male-faces.jpg
https://s-media-cache-ak0.pinimg.com/564x/4b/54/9c/4b549cb826bfe38a6461861cb3a81adb.jpg
https://i.pinimg.com/736x/b7/a2/b7/b7a2b753f806877037c9228ea8ca7166--face-reference-photo-reference.jpg
https://i.pinimg.com/736x/6a/ac/e9/6aace9770f9e6a246402ee7b0f9e293b--man-faces-ding-dong.jpg
https://s-media-cache-ak0.pinimg.com/736x/3e/12/d3/3e12d31531a7eaa87fe661f576f59863.jpg
https://media.gqindia.com/wp-content/uploads/2017/11/andrew-866x956.jpg
https://i.pinimg.com/736x/a4/d7/05/a4d705853e279ca5593fb51a2b8731d2--handso

In [62]:
new_list = []
for i in murl:
    new_list.append(dict(i))

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [67]:
# Webscraper
url = 'https://www.bing.com/images/search?q=male+face&go=Search&qs=ds&form=QBIR&qft=+filterui%3Alicense-L2_L3_L4_L5_L6_L7'

# Query
headers = {'user-agent': 'scraper - school project (bengriffith@outlook.com)'}
response = requests.get(url, headers=headers)

if response.ok:
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    raw_images = soup.select('.content')

In [131]:
i = 0
for link in soup.find_all('a', {'class': 'thumb'}):
    with open('sample' + str(i) + '.png', 'wb') as f:
        response = requests.get(link.get('href'))
        f.write(response.content)
    i += 1